In [1]:
import cv2
import numpy as np

# --- CONFIGURATION ---
# Paramètres pour l'algorithme de Farnebäck (Optimisé pour un compromis vitesse/précision)
FARNEBACK_PARAMS = dict(
    pyr_scale=0.5,   # Échelle de la pyramide (0.5 = pyramide classique)
    levels=3,        # Nombre de niveaux dans la pyramide
    winsize=15,      # Taille de la fenêtre de lissage (plus grand = plus robuste au bruit, mais plus flou)
    iterations=3,    # Itérations par niveau
    poly_n=5,        # Taille du voisinage pour l'expansion polynomiale
    poly_sigma=1.2,  # Écart-type gaussien
    flags=0
)

# Seuil pour considérer qu'il y a mouvement (filtre le bruit de la caméra)
MAGNITUDE_THRESHOLD = 2.0 

# Pas d'échantillonnage pour l'affichage des vecteurs (pour ne pas surcharger l'image)
STEP = 10 

# --- VARIABLES GLOBALES ---
selection_rect = None # (x, y, w, h)
drawing = False
ix, iy = -1, -1

# --- GESTION DE LA SOURIS ---
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, selection_rect
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        selection_rect = None # Reset de la sélection précédente
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            selection_rect = (min(ix, x), min(iy, y), abs(x - ix), abs(y - iy))
            
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        selection_rect = (min(ix, x), min(iy, y), abs(x - ix), abs(y - iy))
        # Éviter les rectangles minuscules
        if selection_rect[2] < 10 or selection_rect[3] < 10:
            selection_rect = None

# --- ANALYSE DU FLOT ---
def analyze_flow_in_roi(flow, roi_gray):
    """
    Extrait les statistiques de mouvement dans la zone d'intérêt.
    """
    # Séparation des composantes horizontale (fx) et verticale (fy)
    fx, fy = flow[..., 0], flow[..., 1]
    
    # Conversion en coordonnées polaires (Magnitude = Vitesse, Angle = Direction)
    mag, ang = cv2.cartToPolar(fx, fy, angleInDegrees=True)
    
    # --- FILTRAGE DU BRUIT (CRUCIAL) ---
    # On ne garde que les pixels qui bougent assez vite
    motion_mask = mag > MAGNITUDE_THRESHOLD
    
    # Si peu de mouvement détecté, on retourne des valeurs nulles
    if np.count_nonzero(motion_mask) < 10:
        return 0.0, 0.0, motion_mask, mag, ang

    # --- CALCUL DES MOYENNES ---
    # Vitesse moyenne des pixels en mouvement uniquement
    avg_speed = np.mean(mag[motion_mask])
    
    # Pour la direction moyenne, on ne peut pas faire la moyenne des angles (problème 0°/360°)
    # On fait la moyenne des vecteurs (fx, fy) puis on convertit
    avg_fx = np.mean(fx[motion_mask])
    avg_fy = np.mean(fy[motion_mask])
    _, avg_angle = cv2.cartToPolar(np.array([avg_fx]), np.array([avg_fy]), angleInDegrees=True)
    
    return avg_speed, avg_angle[0], motion_mask, mag, ang

def get_cardinal_direction(angle):
    """Convertit un angle en degrés en direction textuelle."""
    directions = ["Est (Droite)", "Sud-Est", "Sud (Bas)", "Sud-Ouest", "Ouest (Gauche)", "Nord-Ouest", "Nord (Haut)", "Nord-Est"]
    # L'angle 0 est l'Est dans OpenCV, tourne dans le sens horaire
    idx = int((angle + 22.5) / 45.0) % 8
    return directions[idx]

# --- MAIN LOOP ---
def main():
    global selection_rect
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Erreur: Impossible d'ouvrir la webcam.")
        return

    cv2.namedWindow('Flot Optique - Farneback')
    cv2.setMouseCallback('Flot Optique - Farneback', draw_rectangle)

    prev_gray = None
    prev_roi_frame = None

    print("Contrôles :")
    print("  - SOURIS : Dessiner un rectangle autour de l'objet")
    print("  - 'r'    : Réinitialiser la sélection")
    print("  - 'q'    : Quitter")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Miroir pour une interaction plus naturelle
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        display_frame = frame.copy()

        # Si une zone est sélectionnée
        if selection_rect is not None and selection_rect[2] > 0 and selection_rect[3] > 0:
            x, y, w, h = selection_rect
            
            # Dessin du rectangle de sélection
            cv2.rectangle(display_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
            # Extraction de la ROI (Region of Interest) actuelle
            roi_gray = gray[y:y+h, x:x+w]
            
            # On a besoin de l'image précédente pour calculer le flot
            if prev_roi_frame is not None and prev_roi_frame.shape == roi_gray.shape:
                
                # --- CALCUL DU FLOT OPTIQUE (FARNEBACK) ---
                flow = cv2.calcOpticalFlowFarneback(prev_roi_frame, roi_gray, None, **FARNEBACK_PARAMS)
                
                # --- ANALYSE ---
                speed, angle, mask, mag_map, _ = analyze_flow_in_roi(flow, roi_gray)
                
                # --- VISUALISATION ---
                # 1. Dessiner les vecteurs (lignes vertes)
                # On parcourt la grille avec un pas (STEP)
                for r in range(0, h, STEP):
                    for c in range(0, w, STEP):
                        # On ne dessine que si le mouvement est significatif à cet endroit
                        if mask[r, c]:
                            # Point de départ (dans l'image globale)
                            start_pt = (x + c, y + r)
                            # Vecteur flot
                            fx, fy = flow[r, c]
                            # Point d'arrivée (agrandi x2 pour la visibilité)
                            end_pt = (int(x + c + fx * 2), int(y + r + fy * 2))
                            
                            cv2.arrowedLine(display_frame, start_pt, end_pt, (0, 255, 0), 1, tipLength=0.3)

                # 2. Afficher les statistiques
                direction_text = get_cardinal_direction(angle) if speed > 0.5 else "Stable"
                
                # Barre d'info
                info_text = f"Vitesse: {speed:.2f} px/frame | Direction: {direction_text} ({angle:.0f} deg)"
                cv2.putText(display_frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                
                # 3. Code couleur de l'état
                if speed > 10:
                    status = "MOUVEMENT RAPIDE"
                    color = (0, 0, 255) # Rouge
                elif speed > 0.5:
                    status = "MOUVEMENT LENT"
                    color = (0, 255, 0) # Vert
                else:
                    status = "IMMOBILE"
                    color = (200, 200, 200) # Gris
                    
                cv2.putText(display_frame, status, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Mise à jour pour la frame suivante
            prev_roi_frame = roi_gray.copy()
        else:
            prev_roi_frame = None
            cv2.putText(display_frame, "Dessinez un rectangle avec la souris", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        if drawing and selection_rect:
             x, y, w, h = selection_rect
             cv2.rectangle(display_frame, (x, y), (x + w, y + h), (0, 255, 255), 1)

        cv2.imshow('Flot Optique - Farneback', display_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('r'):
            selection_rect = None
            prev_roi_frame = None

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Contrôles :
  - SOURIS : Dessiner un rectangle autour de l'objet
  - 'r'    : Réinitialiser la sélection
  - 'q'    : Quitter


C:\Users\moura\AppData\Local\Temp\ipykernel_52180\3878625284.py:82: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  idx = int((angle + 22.5) / 45.0) % 8


TypeError: unsupported format string passed to numpy.ndarray.__format__